In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pre_processing as pp
import analysis as anl
import pca

category = 'kitchen_&_housewares'

hNeg = True #if true, add negative bigrams for negative reviews
noun = False #if true, add nouns

X, Y, vocabulary = pp.bow(category, hNeg, noun)

print("Vocabulário possui " + str(len(vocabulary)) + " palavras!")

Vocabulário possui 5023 palavras!


In [4]:
# semente usada na randomizacao dos dados.
randomSeed = 10 

# gera os indices aleatorios que irao definir a ordem dos dados
idx_perm = np.random.RandomState(randomSeed).permutation(range(len(Y)))

# ordena os dados de acordo com os indices gerados aleatoriamente
X2, Y2 = X[idx_perm, :], Y[idx_perm]

pTrain = 0.8

train_index, test_index = anl.stratified_holdOut(Y, pTrain)

Xtrain, Xval = X2[train_index, :], X2[test_index, :];
Ytrain, Yval = Y2[train_index], Y2[test_index];

In [5]:
Xtrain, new_vocabulary, index = pp.chi2(Xtrain, Ytrain, vocabulary)
Xval = Xval[:, index]

#Converte matrizes esparsas para np arrays, para os cálculos da regressão logística
Xtrain = Xtrain.toarray()
Xval = Xval.toarray()

/home/sy/Downloads/MachineLearning/am/pre_processing.py:203: RuntimeWarning: divide by zero encountered in true_divide
  chisq /= f_exp


In [4]:
print("Número de features antes do chi-quadrado: " + str(len(vocabulary)))
print("----------------------------------------")
print("Número de features após chi-quadrado: " + str(len(new_vocabulary)))
print(new_vocabulary)

Número de features antes do chi-quadrado: 5023
----------------------------------------
Número de features após chi-quadrado: 195
['not', 'wooden', 'sharpen', 'told', 'beauti', 'rais', 'comfort', 'spout', 'tri', ('not', 'get'), ('not', 'good'), 'long', 'arriv', 'fabul', 'plastic', 'outstand', 'bought', 'purchas', 'need', 'fish', 'bad', 'broken', 'less', 'difficult', 'sharp', 'fantast', 'die', 'quick', 'bark', 'go', 'bread', 'warp', 'aw', 'add', ('not', 'even'), 'depend', 'whisk', 'return', ('not', 'know'), 'bare', ('not', 'use'), 'kitchenaid', 'danger', ('not', 'seem'), 'heavi', 'even', 'hear', ('not', 'worth'), 'start', ('not', 'last'), 'rememb', ('not', 'give'), ('not', 'purchas'), ('not', 'clean'), 'slow', 'zest', 'buy', 'amaz', 'come', 'save', ('not', 'tri'), 'perfect', ('not', 'go'), 'far', 'warn', 'send', 'love', 'unfortun', 'everywher', 'effici', 'beat', ('not', 'not'), 'end', 'easi', 'no', 'quiet', 'kitchen', 'crush', 'power', 'base', 'imposs', 'easier', ('not', 'sure'), 'nice'

In [6]:
def sigmoid(z):
    
    """
    Calcula a funcao sigmoidal  
    """
    
    if isinstance(z, int):
        g = 0
    
    # se z não é um inteiro, significa que é um array e inicia com a dimensão do array
    else:
        g = np.zeros( z.shape );

    g = 1/(1+np.exp(-z))
  
    return g

In [20]:
def funcaoCustoReg(theta, X, Y, lambda_reg):
    
    """
    Calcula o custo da regressao logística
    """
    
    m = len(Y) #numero de exemplos de treinamento

    J = 0
    grad = np.zeros( len(theta) )
    
    # parâmetro de tolerância para a função sigmoide. 1-htheta não pode ser 
    # menor que eps para evitar erro de precisão numérica
    eps = 1e-15
    
    z = np.dot(X, theta) # z = (theta * X).sum(axis=1)
    h_theta = sigmoid(z) 
    cost = -Y * np.log(h_theta+eps) - (1 - Y) * np.log(1 - h_theta + eps)
    J = 1/m * cost.sum()
    
    grad = 1/m *(X.T * (h_theta - Y)).sum(axis=1)
                      
    return J, grad


In [21]:
import scipy
import scipy.optimize 

def treinamento(X, Y, lambda_reg, iteracoes):
    
    # se for vazio, retorna None 
    if len(Y)==0:
        return None
    
    m, n = X.shape # m = qtde de objetos e n = qtde de atributos por objeto
    
    theta = np.zeros(n) # Inicializa parâmetros que serao ajustados
    
    # minimiza a funcao de custo
    result = scipy.optimize.minimize(fun=funcaoCustoReg, x0=theta, args=(X, Y, lambda_reg),  
                method='BFGS', jac=True, options={'maxiter': iteracoes, 'disp':False})

    # coleta os thetas retornados pela função de minimização
    theta = result.x
    
    return theta

In [38]:
def predicao(X, theta):
    
    """
    Prediz se a entrada pertence a classe 0 ou 1 usando o parametro
    theta obtido pela regressao logistica
   
    """
    m = X.shape[0]   
#    Ypred = np.zeros(m, dtype="int")
    
#    h = sigmoid( np.dot(X,theta) )
    
#    for i in range(m):
#        if h[i] >= 0.5:
#            Ypred[i] = 1
#        else:
#            Ypred[i] = 0
            
#    acuracia = np.sum(np.array(Ypred[0])==Yval)/len(Yval)
        
#    return Ypred, acuracia
    p = np.zeros(m, dtype=int) 
    z = np.dot(X, theta)
    h_theta = sigmoid(z) 
    
    for i in range(m):
        if h_theta[i] >= 0.5:
            p[i] = 1 
    
    return p

In [29]:
theta = treinamento(Xtrain, Ytrain, 25, 50)

In [30]:
Ypred = predicao(Xval, theta)
acuracia = (np.sum(Ypred==Yval)/len(Yval)) * 100
print("Acurácia é "+ str(acuracia))

NameError: name 'm' is not defined

In [ ]:
classes = np.unique(Y)
auxResults = anl.relatorioDesempenho(Yval, Ypred, classes, imprimeRelatorio=True)

In [ ]:
def curva_aprendizado(Xtrain, Ytrain, Xval, Yval):
   
    """
    Funcao usada gerar a curva de aprendizado.
  
    Parametros
    ----------
  
    X : matriz com os dados de treinamento
  
    Y : vetor com as classes dos dados de treinamento
  
    Xval : matriz com os dados de validação
  
    Yval : vetor com as classes dos dados de validação
  
    """

    # inicializa as listas que guardarao a performance no treinamento e na validacao
    
    perf_train = []
    perf_val = []

    classes = np.unique(Y)
    
    for i in range(10, len(Y)):
        
        theta = treinamento(Xtrain[:i], Ytrain[:i], 25, 50)
        
        Ypred, acuracia = predicao(Xtrain[:i], theta, Ytrain[:i])
        perf_train.append(acuracia)

        Ypred, acuracia = predicao(Xval, theta, Yval)
        perf_val.append(acuracia)

    ##################################################################################
       
    # Define o tamanho da figura 
    plt.figure(figsize=(20,12))

    # Plota os dados
    plt.plot(perf_train, color='blue', linestyle='-', linewidth=1.5, label='Treino') 
    plt.plot(perf_val, color='red', linestyle='-', linewidth=1.5, label='Validação')

    # Define os nomes do eixo x e do eixo y
    plt.xlabel(r'# Qtd. de dados de treinamento',fontsize='x-large') 
    plt.ylabel(r'Acuracia',fontsize='x-large') 

    # Define o título do gráfico
    plt.title(r'Curva de aprendizado', fontsize='x-large')

    # Acrescenta um grid no gráfico
    plt.grid(axis='both')

    # Plota a legenda
    plt.legend()
    
    plt.show()



In [ ]:
curva_aprendizado(Xtrain, Ytrain, Xval, Yval)

In [39]:
def gridSearch(X, Y ,Xval, Yval, lambda_reg, iteracoes) :
    bestLambda = lambda_reg[0]
    bestAcc = 0
    for l in lambda_reg :
        theta = treinamento(X,Y, l, iteracoes)
        Ypred = predicao(Xval, theta)
        acuracia = (np.sum(Ypred==Yval)/len(Yval)) * 100
        print("Acurácia é "+ str(acuracia))
        if bestAcc < acuracia :
            bestLambda = l
            bestAcc = acuracia
        
    print("Best Lambda = ", bestLambda)
    print("Acc = ", bestAcc)
    return bestLambda

In [40]:
lambda_reg = [0.001, 0.01, 0.1, 1, 0.05, 0.5, 5, 10, 50, 100, 500]
l = gridSearch(Xtrain, Ytrain, Xval, Yval, lambda_reg, 100)

Acurácia é 90.0
Acurácia é 90.0
Acurácia é 90.0
Acurácia é 90.0
Acurácia é 90.0
Acurácia é 90.0
Acurácia é 90.0
Acurácia é 90.0
Acurácia é 90.0
Acurácia é 90.0
Acurácia é 90.0
Best Lambda =  0.001
Acc =  90.0
